In [1]:
import torch
import torchvision.transforms as tvt
from torch.utils.data import DataLoader
from tqdm import tqdm
from pytorch_ood.dataset.img import Textures
from pytorch_ood.detector import OpenMax
from pytorch_ood.utils import OODMetrics, ToUnknown
import shutil
import pandas as pd
import numpy as np
import os
torch.manual_seed(1234)
device = "cuda:2"

/home/phucdt/anaconda3/envs/bio/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/phucdt/anaconda3/envs/bio/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## data

In [3]:
# load dev set
devdf = pd.read_csv("xinwang_vocoders/data/voc.v4/protocol.txt", sep=" ", header=None)
devdf.columns = ["path", "subset","label"]

In [4]:
# load out-of-distribution set
melgan_dir = 'xinwang_vocoders/data/voc.v4/MelGAN'
file_list = os.listdir(melgan_dir)

# make dataframe for out-of-distribution set
outdf = pd.DataFrame()
outdf['path'] = file_list
outdf['subset'] = 'test'
outdf['label'] = 'unknown'
outdf['path'] = outdf['path'].apply(lambda x: os.path.join('MelGAN', x))

FileNotFoundError: [Errno 2] No such file or directory: 'xinwang_vocoders/data/voc.v4/MelGAN'

In [11]:
outdf.head()

,path,subset,label
0,MelGan/LA_D_3727888.wav,test,unknown
1,MelGan/LA_T_4565832.wav,test,unknown
2,MelGan/LA_D_6446182.wav,test,unknown
3,MelGan/LA_T_1407047.wav,test,unknown
4,MelGan/LA_D_5542285.wav,test,unknown


In [16]:
testdf = pd.concat([devdf[devdf['subset'] == 'dev'], outdf[:2028]])

In [17]:
testdf.label.value_counts()

bonafide            2074
hn-sinc-nsf         2072
waveglow            2062
unknown             2028
hifigan             2026
hn-sinc-nsf-hifi    2023
Name: label, dtype: int64

In [22]:
# write to protocol_test.txt
testdf['subset'] = 'dev'
testdf.to_csv('xinwang_vocoders/data/voc.v4/protocol_test.txt', sep=" ", header=None, index=False)

In [52]:
# load in_the_wild set
protocol = pd.read_csv("/dataa/Dataset/cnsl_real_fake_audio/protocols/protocol_in_the_wild.txt", sep=" ", header=None)
protocol.columns = ["path", "subset","label"]
protocol_bona = protocol[protocol['label'] == 'bonafide']

In [55]:
for index, row in tqdm(protocol_bona.iterrows(), total=len(protocol_bona)):
    path = os.path.join("traindata/jul6/",row['path'])
    shutil.copy(path, "traindata/jul6/in_the_wild_bona/")
    

100%|██████████| 19963/19963 [03:11<00:00, 104.23it/s]


## Load model and data to fit the OpenMax

In [2]:
from datautils.vocv4 import genList, Dataset_for, Dataset_for_eval
from model.wav2vec2_resnet import Model
import yaml

In [3]:
# load config
config = yaml.load(open("configs/wav2vec2_resnet.yaml", "r"), Loader=yaml.FullLoader)
# load model
model = Model(args=config['model'], device=device, emb=False)
# load state dict
model.load_state_dict(torch.load('out/model_weighted_CCE_100_8_1e-06_wav2vec2_resnet/epoch_31.pth',map_location=device))
model = model.to(device)

In [4]:
# load train data
d_label_trn,file_train = genList( dir_meta =  os.path.join('xinwang_vocoders/data/voc.v4/','protocol.txt'),is_train=True,is_eval=False,is_dev=False)
print('no. of training trials',len(file_train))
train_set=Dataset_for(config,list_IDs = file_train,labels = d_label_trn,base_dir = os.path.join('xinwang_vocoders/data/voc.v4/'),algo=1)
train_loader = DataLoader(train_set, batch_size=8,num_workers=8, shuffle=True,drop_last = True)
del train_set,d_label_trn

no. of training trials 15383


In [5]:
# Load test data
d_label_dev, file_dev = genList(dir_meta = os.path.join('xinwang_vocoders/data/voc.v4/','protocol_test.txt'),is_train=False,is_eval=False, is_dev=True)
    
print('no. of validation trials',len(file_dev))
    
dev_set = Dataset_for(config,list_IDs = file_dev,
	labels = d_label_dev,
	base_dir = os.path.join('xinwang_vocoders/data/voc.v4/'),algo=1)
test_loader = DataLoader(dev_set, batch_size=8,num_workers=8, shuffle=False)
# del dev_set,d_label_dev


no. of validation trials 12285


In [8]:
detector = OpenMax(model, alpha=5)
detector.fit(train_loader, device=device)

In [9]:
metrics = OODMetrics()
for x, y in test_loader:
    score = detector(x.to(device))
    # print(score, y)
    metrics.update(score, y)


In [10]:
scores = metrics.buffer.get('scores')
labels = metrics.buffer.get('labels')

In [11]:
scores, scores_idx = torch.sort(scores, stable=True)
labels = labels[scores_idx]

In [14]:
precision, recall, threshold = binary_precision_recall_curve(scores, labels)
print(len(threshold))


12286


## MaxSoftmax


In [2]:
from torchmetrics.functional.classification import (
    binary_auroc,
    binary_precision_recall_curve,
    binary_roc,
)

In [3]:
from maxsoftmax import MaxSoftmax
detector = MaxSoftmax(model)
metrics = OODMetrics()
preds = []
for x, y in test_loader:
    score, pred = detector(x.to(device))
    metrics.update(score, y)
    # print(pred)
    preds.extend(pred.tolist())

NameError: name 'model' is not defined

In [22]:
testdf = pd.read_csv("/dataa/phucdt/vocodetect/testdf_aasist_jul6_balanceweight_59_wavefake.txt", sep=" ")
testdf.head

<bound method NDFrame.head of                                                   path subset     label  \
0    generated_audio/ljspeech_parallel_wavegan/LJ01...   test     spoof   
1    generated_audio/common_voices_prompts_from_con...   test     spoof   
2    generated_audio/ljspeech_multi_band_melgan/LJ0...   test     spoof   
3    generated_audio/ljspeech_multi_band_melgan/LJ0...   test     spoof   
4     generated_audio/ljspeech_waveglow/LJ042-0066.wav   test     spoof   
..                                                 ...    ...       ...   
995                            LJSpeech/LJ037-0117.wav   test  bonafide   
996                            LJSpeech/LJ029-0081.wav   test  bonafide   
997                            LJSpeech/LJ001-0121.wav   test  bonafide   
998                            LJSpeech/LJ035-0118.wav   test  bonafide   
999                            LJSpeech/LJ015-0170.wav   test  bonafide   

        score  pred  
0   -0.999410     0  
1   -0.999962     0  
2  

In [23]:
from datautils.vocv4 import LABELS
testdf['label'] = testdf['label'].apply(lambda x: LABELS[x] if x in LABELS else 5)
testdf.head

<bound method NDFrame.head of                                                   path subset  label  \
0    generated_audio/ljspeech_parallel_wavegan/LJ01...   test      5   
1    generated_audio/common_voices_prompts_from_con...   test      5   
2    generated_audio/ljspeech_multi_band_melgan/LJ0...   test      5   
3    generated_audio/ljspeech_multi_band_melgan/LJ0...   test      5   
4     generated_audio/ljspeech_waveglow/LJ042-0066.wav   test      5   
..                                                 ...    ...    ...   
995                            LJSpeech/LJ037-0117.wav   test      0   
996                            LJSpeech/LJ029-0081.wav   test      0   
997                            LJSpeech/LJ001-0121.wav   test      0   
998                            LJSpeech/LJ035-0118.wav   test      0   
999                            LJSpeech/LJ015-0170.wav   test      0   

        score  pred  
0   -0.999410     0  
1   -0.999962     0  
2   -0.999981     0  
3   -0.999962    

In [24]:
# modify the prediction based on the threshold
def fix_pred(row):
    if row['score'] >= -0.99994:
        return 5
    else:
        return row['pred']
testdf['pred_fix'] = testdf.apply(fix_pred, axis=1)
# testdf.head

In [7]:
testdf[testdf['label']==0]

,path,subset,label,score,pred,pred_fix
4,wav/LA_T_8847275.wav,dev,0,-0.999427,0,0
9,wav/LA_D_7605047.wav,dev,0,-0.999375,0,0
10,wav/LA_T_6708956.wav,dev,0,-0.999421,0,0
19,wav/LA_D_6509046.wav,dev,0,-0.997738,0,0
22,wav/LA_T_7696598.wav,dev,0,-0.997754,0,0
...,...,...,...,...,...,...
10219,wav/LA_D_6089259.wav,dev,0,-0.998612,0,0
10225,wav/LA_T_6426400.wav,dev,0,-0.998655,0,0
10231,wav/LA_D_7267402.wav,dev,0,-0.998611,0,0
10240,wav/LA_D_3891696.wav,dev,0,-0.998850,0,0


In [25]:
from torcheval.metrics.functional import multiclass_accuracy, multiclass_confusion_matrix
# multi-class accuracy
input = torch.tensor(testdf['pred_fix'])
target = torch.tensor(testdf['label'])
print(multiclass_accuracy(input, target))
multiclass_confusion_matrix(input, target, num_classes=6)

tensor(0.5070)


tensor([[263,   0,   0,   0,   0, 237],
        [  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0],
        [256,   0,   0,   0,   0, 244]])

In [26]:
testdf[(testdf['pred_fix'] == 5) & (testdf['label'] == 0)]

,path,subset,label,score,pred,pred_fix
503,LJSpeech/LJ025-0154.wav,test,0,-0.981259,0,5
504,LJSpeech/LJ035-0124.wav,test,0,-0.999136,0,5
508,LJSpeech/LJ005-0213.wav,test,0,-0.998954,0,5
513,LJSpeech/LJ023-0084.wav,test,0,-0.999749,0,5
517,LJSpeech/LJ028-0223.wav,test,0,-0.999874,0,5
...,...,...,...,...,...,...
993,LJSpeech/LJ010-0054.wav,test,0,-0.999809,0,5
994,LJSpeech/LJ040-0080.wav,test,0,-0.999070,0,5
995,LJSpeech/LJ037-0117.wav,test,0,-0.998158,0,5
996,LJSpeech/LJ029-0081.wav,test,0,-0.999773,0,5


In [27]:
# binary accuracy
def pred_bi(row):
    if row['pred_fix'] == 0:
        return 0
    else:
        return 1
def label_bi(row):
    if row['label'] == 0:
        return 0
    else:
        return 1
testdf['pred_bi'] = testdf.apply(pred_bi, axis=1)
testdf['bi_label'] = testdf.apply(label_bi, axis=1)
testdf.head

<bound method NDFrame.head of                                                   path subset  label  \
0    generated_audio/ljspeech_parallel_wavegan/LJ01...   test      5   
1    generated_audio/common_voices_prompts_from_con...   test      5   
2    generated_audio/ljspeech_multi_band_melgan/LJ0...   test      5   
3    generated_audio/ljspeech_multi_band_melgan/LJ0...   test      5   
4     generated_audio/ljspeech_waveglow/LJ042-0066.wav   test      5   
..                                                 ...    ...    ...   
995                            LJSpeech/LJ037-0117.wav   test      0   
996                            LJSpeech/LJ029-0081.wav   test      0   
997                            LJSpeech/LJ001-0121.wav   test      0   
998                            LJSpeech/LJ035-0118.wav   test      0   
999                            LJSpeech/LJ015-0170.wav   test      0   

        score  pred  pred_fix  pred_bi  bi_label  
0   -0.999410     0         5        1         1  
1  

In [28]:
from torcheval.metrics.functional import binary_accuracy, binary_confusion_matrix
print(binary_accuracy(torch.tensor(testdf['pred_bi']), torch.tensor(testdf['bi_label'])))
print(binary_confusion_matrix(torch.tensor(testdf['pred_bi']), torch.tensor(testdf['bi_label'])))

tensor(0.5070)
tensor([[263, 237],
        [256, 244]])


In [25]:
testdf[(testdf['label'] == 1) & (testdf['pred_bi'] == 1)]

,path,subset,label,score,pred,pred_fix,pred_bi
7,wav/hifigan_LA_T_9219481.wav,dev,1,-0.999566,1,1,1
11,wav/hifigan_LA_T_9203438.wav,dev,1,-0.998440,1,1,1
13,wav/hifigan_LA_T_6240107.wav,dev,1,-0.998435,1,1,1
14,wav/hifigan_LA_T_9918657.wav,dev,1,-0.998474,1,1,1
16,wav/hifigan_LA_D_2412960.wav,dev,1,-0.998258,1,1,1
...,...,...,...,...,...,...,...
10220,wav/hifigan_LA_D_3467485.wav,dev,1,-0.998140,1,1,1
10221,wav/hifigan_LA_D_7879955.wav,dev,1,-0.997882,1,1,1
10238,wav/hifigan_LA_D_6986371.wav,dev,1,-0.998709,1,1,1
10249,wav/hifigan_LA_D_8899535.wav,dev,1,-0.998937,1,1,1


In [26]:
scores = torch.Tensor(testdf['score'].values)
labels = torch.Tensor(testdf['label'].values)

In [9]:
scores = metrics.buffer.get('scores')
labels = metrics.buffer.get('labels')

In [14]:
def compute_det_curve(target_scores, nontarget_scores):

    n_scores = target_scores.size + nontarget_scores.size
    all_scores = np.concatenate((target_scores, nontarget_scores))
    labels = np.concatenate((np.ones(target_scores.size), np.zeros(nontarget_scores.size)))

    # Sort labels based on scores
    indices = np.argsort(all_scores, kind='mergesort')
    labels = labels[indices]

    # Compute false rejection and false acceptance rates
    tar_trial_sums = np.cumsum(labels)
    nontarget_trial_sums = nontarget_scores.size - (np.arange(1, n_scores + 1) - tar_trial_sums)

    frr = np.concatenate((np.atleast_1d(0), tar_trial_sums / target_scores.size))  # false rejection rates
    far = np.concatenate((np.atleast_1d(1), nontarget_trial_sums / nontarget_scores.size))  # false acceptance rates
    thresholds = np.concatenate((np.atleast_1d(all_scores[indices[0]] - 0.001), all_scores[indices]))  # Thresholds are the sorted scores

    return frr, far, thresholds

def calculate_confusion_matrix(target_scores, nontarget_scores, threshold):
    """
    Calculate the confusion matrix for a given threshold.
    return: tp, tn, fp, fn
    """
    tp = np.sum(target_scores > threshold)
    tn = np.sum(nontarget_scores <= threshold)
    fn = np.sum(target_scores <= threshold)
    fp = np.sum(nontarget_scores > threshold)
    return tp, tn, fp, fn

def compute_eer(target_scores, nontarget_scores):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    frr, far, thresholds = compute_det_curve(target_scores, nontarget_scores)
    abs_diffs = np.abs(frr - far)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((frr[min_index], far[min_index]))
    return eer, thresholds[min_index]

In [27]:
known_scores = scores[labels != 0]
unknown_scores = scores[labels == 0]

In [28]:
eer, th = compute_eer(unknown_scores.cpu().numpy(), known_scores.cpu().numpy())
tp, tn, fp, fn = calculate_confusion_matrix(unknown_scores.cpu().numpy(), known_scores.cpu().numpy(), th)

print("EER: {:.2f}%, threshold: {:.4f}".format(eer * 100, th))
print("TP: {}, TN: {}, FP: {}, FN: {}".format(tp, tn, fp, fn))

EER: 55.59%, threshold: -0.9987
TP: 921, TN: 4534, FP: 5677, FN: 1153


In [29]:
from torcheval.metrics import BinaryAccuracy
metric = BinaryAccuracy(threshold=-0.9987)
metric.update(scores, labels)
print(metric.compute())



tensor(0.1796)


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 2; 22.19 GiB total capacity; 16.09 GiB already allocated; 12.44 MiB free; 16.71 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF